In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE211378"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE211378"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE211378.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE211378.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE211378.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Whole Blood profiling of COVID convalescent and Healthy donors with nCounter"
!Series_summary	"This study investigated the cellular immune response in people who had recovered from SARS-CoV2 infection (COVID-19)."
!Series_overall_design	"264 Whole Blood samples from 160 COVID convalescent donors, and 40 from Healthy donors."
Sample Characteristics Dictionary:
{0: ['sample.id: host response panel  1 wb', 'sample.id: host response panel 2 wb', 'sample.id: host response panel 3 wb', 'sample.id: host response panel 4 wb', 'sample.id: host response panel plate 25 wb 11292020', 'sample.id: host response panel plate 5 wb 11 19 2020', 'sample.id: host response panel plate 6 wb 11192020', 'sample.id: host response panel plate 7 wb 11 20 2020', 'sample.id: host response panel plate 8 wb 11202020', 'sample.id: host response panel plate 26 wb 11302020', 'sample.id: host response panel plate 9 wb 11212020', 'sample.id: host response panel plate 10 wb 11212020'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # The background suggests this is gene expression profiling data

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# There are no clear clinical data columns in the sample characteristics
# From the background information: "264 Whole Blood samples from 160 COVID convalescent donors, 
# and 40 from Healthy donors." implies there is COVID-19 status information
# However, it's not available in the sample characteristics dictionary
trait_row = None  # No clear trait information in characteristics
age_row = None    # No age information found
gender_row = None # No gender information found

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # Function to convert COVID-19 status, though not used in this dataset
    if value is None:
        return None
    
    value = value.lower().split(': ')[-1].strip()
    if 'covid' in value or 'convalescent' in value or 'infected' in value or 'positive' in value:
        return 1
    elif 'healthy' in value or 'control' in value or 'negative' in value:
        return 0
    return None

def convert_age(value):
    # Function to convert age to continuous values, though not used in this dataset
    if value is None:
        return None
    
    try:
        # Extract the value after the colon and convert to float
        age_value = value.split(': ')[-1].strip()
        return float(age_value)
    except (ValueError, AttributeError):
        return None

def convert_gender(value):
    # Function to convert gender to binary, though not used in this dataset
    if value is None:
        return None
    
    value = value.lower().split(': ')[-1].strip()
    if value in ['female', 'f', 'woman']:
        return 0
    elif value in ['male', 'm', 'man']:
        return 1
    return None

# 3. Save Metadata
# Initial validation - check if the dataset has both gene and trait data
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=(trait_row is not None)
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE211378/GSE211378_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE211378/GSE211378_series_matrix.txt.gz
Found the matrix table marker at line 84
Gene data shape: (773, 304)
First 20 gene/probe identifiers:
['ACE', 'ACKR2', 'ACKR3', 'ACKR4', 'ACOX1', 'ACSL1', 'ACSL3', 'ACSL4', 'ACVR1', 'ADAR', 'ADGRE5', 'ADGRG3', 'ADORA2A', 'AGT', 'AHR', 'AIF1', 'AIM2', 'AKT1', 'AKT2', 'AKT3']


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers based on biomedical knowledge
# The identifiers appear to be standard human gene symbols (ACE, ACKR2, AKT1, etc.)
# These are official HGNC gene symbols and do not need to be mapped

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create empty clinical data for record-keeping purposes
empty_clinical_df = pd.DataFrame()

# Create directory for clinical data output
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)

# Save the empty clinical features
empty_clinical_df.to_csv(out_clinical_data_file)
print(f"Empty clinical features saved to {out_clinical_data_file}")

# Since we don't have trait data, we can't perform:
# - linking with gene data
# - handling missing values
# - determining trait bias
# - creating usable linked data

# For validation purposes, mark the dataset as unusable due to lack of trait data
is_trait_available = False  # No trait data available as determined in Step 2
is_biased = True  # Mark as biased (unusable) since we can't analyze trait-related bias

# Validate and save cohort info - mark as unusable due to missing trait data
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=empty_clinical_df,  # Use empty DataFrame as placeholder
    note="Dataset contains gene expression data but lacks necessary trait information for COVID-19 analysis."
)

print("Linked data not created or saved due to missing trait information.")

Gene data shape after normalization: (758, 304)


Normalized gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE211378.csv
Empty clinical features saved to ../../output/preprocess/COVID-19/clinical_data/GSE211378.csv
Abnormality detected in the cohort: GSE211378. Preprocessing failed.
Linked data not created or saved due to missing trait information.
